In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sqlalchemy import create_engine
import urllib
import pyodbc
import requests
import pgeocode
from time import sleep

In [ ]:
# Download from here https://www.colorado.gov/pacific/enforcement/med-licensed-facilities

In [ ]:
# Just so I see the whole DataFrame
pd.options.display.max_columns = 50
pd.options.display.min_rows = 100
pd.options.display.max_rows = 100

In [ ]:
# Insert Today's Month and Year for later in the code
# Day doesn't matter, so it can be kept as 1

month_year = dt.date(2022, 2, 1)

# Make sure Month and Year match before continuing code

this_month = month_year.strftime('%B')
this_month_num = month_year.strftime('%m')
this_year = month_year.strftime('%Y')

### Create SQL Connection

In [ ]:
# These will be used later

In [ ]:
# # Local Engine
# driver = "ODBC Driver 17 for SQL Server"
# engine = create_engine(f'mssql://LAPTOP-E6QKON1L/co_cannabis?driver={driver}')
# engine_con = engine.connect()

In [ ]:
# Bespoke Engine
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=co_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine_con = engine.connect()

## Code to Clean Data

In [ ]:
# This function reads and combines the two sheets from the excel files

def read_med_and_retail(excel_file):
    medical_df = pd.read_excel(excel_file, sheet_name='Medical')
    retail_df = pd.read_excel(excel_file, sheet_name='Retail')
    
    if len(medical_df.columns) > 3 and len(retail_df.columns) > 3:
        total_df = retail_df.append(medical_df).reset_index().drop(columns='index')
    elif len(medical_df.columns) > 3 and len(retail_df.columns) <= 3:
        total_df = medical_df
    elif len(medical_df.columns) <= 3 and len(retail_df.columns) > 3:
        total_df = retail_df
    
    return total_df

In [ ]:
# This funciton will clean the dataframes once they are created 

def clean_df(dataframe, license_type):
    dataframe = dataframe.drop(dataframe.columns[dataframe.columns.str.contains('Unnamed', case=False)], axis=1)
    dataframe = dataframe[dataframe['LicenseNumber'].notnull()]
    
    dataframe['LicenseType'] = dataframe['LicenseNumber'].str.contains('R').replace(True, f"Retail {license_type}").replace(False, f"Medical {license_type}")
    return dataframe

## Read in and Clean the DFs

### Cultivation

In [ ]:
cultivation_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Cultivations.xlsx')

In [ ]:
cultivation_df = clean_df(cultivation_df, 'Cultivation')

In [ ]:
cultivation_df.head()

### Delivery

In [ ]:
delivery_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Delivery.xlsx')

In [ ]:
delivery_df = clean_df(delivery_df, 'Delivery')

In [ ]:
delivery_df.head()

### Hospitality

In [ ]:
hospitality_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Hospitality.xlsx')

In [ ]:
hospitality_df = clean_df(hospitality_df, 'Hospitality')

In [ ]:
hospitality_df.head()

### Operators

In [ ]:
operators_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Operators.xlsx')

In [ ]:
operators_df = clean_df(operators_df, 'Operator')

In [ ]:
operators_df.head()

### Product Manufacturers

In [ ]:
product_manufacturers_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Product Manufacturers.xlsx')

In [ ]:
product_manufacturers_df = clean_df(product_manufacturers_df, 'Product Manufacturer')

In [ ]:
product_manufacturers_df.head()

### Research Development Cultivations

In [ ]:
research_development_cultivation_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Research _ Development Cultivations.xlsx')

In [ ]:
research_development_cultivation_df = clean_df(research_development_cultivation_df, 'Research Development Cultivation')

In [ ]:
research_development_cultivation_df.head()

### Stores

In [ ]:
stores_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Stores.xlsx')

In [ ]:
stores_df = clean_df(stores_df, 'Store')

In [ ]:
stores_df.head()

### Testing Facilities

In [ ]:
testing_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Testing Facilities.xlsx')

In [ ]:
testing_df.rename(columns={'License # ': 'LicenseNumber'}, inplace=True)

In [ ]:
testing_df

In [ ]:
testing_df = clean_df(testing_df, 'Testing')

In [ ]:
testing_df.head()

### Transporters

In [ ]:
transporters_df = read_med_and_retail(f'license_repository/{this_year}-{this_month_num}/Transporters.xlsx')

In [ ]:
transporters_df = clean_df(transporters_df, 'Transporter')

In [ ]:
transporters_df.head()

## Combine the DataFrames

In [ ]:
cultivation_df.columns

In [ ]:
delivery_df.columns

In [ ]:
hospitality_df.columns

In [ ]:
operators_df.columns

In [ ]:
product_manufacturers_df.columns

In [ ]:
research_development_cultivation_df.columns

In [ ]:
stores_df.columns

In [ ]:
testing_df.columns

In [ ]:
transporters_df.columns

In [ ]:
combined_df = cultivation_df.append([
        delivery_df, operators_df, 
        product_manufacturers_df, 
        research_development_cultivation_df, stores_df,
# #         testing_df,
        hospitality_df, transporters_df
            ]).reset_index().drop(columns='index')

In [ ]:
combined_df['FacilityType'].unique()

In [ ]:
combined_df.FacilityType.value_counts()

In [ ]:
hospitality_df

## Rename and Add Columns to Match df_final CSV

In [ ]:
# This is a quick rename of all of the 

df_all = combined_df.rename(columns={
            'LicenseNumber': 'license_number',
            'FacilityType': 'license_category',
            'LicenseType': 'license_description',
            'FacilityName': 'name_legal',
            'DBA': 'name_dba',
            'City': 'contact_city',
            'ZipCode': 'contact_zip',
            'DateUpdated': 'date_uploaded'
                    })

In [ ]:
# This will create a binary column to confirm if the license is still current, i.e. is still present in the newly updated repos 
df_all['is_current'] = 1

### Quick Bit Of Name Cleaning

In [ ]:
# Just a quick space removal from both sides of both names
df_all['name_legal'] = df_all['name_legal'].str.rstrip()
df_all['name_legal'] = df_all['name_legal'].str.lstrip()
df_all['name_dba'] = df_all['name_dba'].str.rstrip()
df_all['name_dba'] = df_all['name_dba'].str.lstrip()

### name_legal_clean

In [ ]:
# This adds and cleans the name_legal_clean column

# df_all['name_legal_clean'] = df_all['name_legal']

# pd.set_option('mode.chained_assignment', None)

# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.lower() 
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, inc.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\binc.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\binc\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, llc.\b", "") 
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, llc\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\b, l.l.c.\b", "") 
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bllc\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bcorp.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bcorp\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bcorporation\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bco.\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(r"\bco\b", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(",", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace(".", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace("'", "")
# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.strip()

In [ ]:
# 1-11 kept auto-changing to 11-Jan in Excel which was driving me crazy
# The company seems to use both - and : in it's titles, so this should be a clean way to fix it
# They also use ONE:ELEVEN as their name in their logo, so that might also work

# df_all['name_legal_clean'] = df_all['name_legal_clean'].str.replace('1-11', "1:11")

### roll_up

In [ ]:
# Add these two columns so they can be fiddled with in the CSV

df_all['roll_up_id'] = ''
df_all['company_roll_up'] = ''

## Contact Info

In [ ]:
# This is creating a dataframe of the data that will be transfered for the co_contact database

contact_transfer = pd.DataFrame(df_all.license_number.unique(), columns=['license_number'])

contact_transfer['name_legal'] = ''

contact_transfer['contact_website'] = ''
contact_transfer['contact_email'] = ''
contact_transfer['contact_phone'] = ''

contact_transfer['contact_street'] = ''
contact_transfer['contact_city'] = ''
contact_transfer['contact_state'] = 'CO'
contact_transfer['contact_zip'] = ''

### Add roll_up_ids and contact info from SQL

In [ ]:
# This adds the already calculated roll_up_ids from sql

In [ ]:
sql_roll = pd.read_sql('co_roll', con=engine_con)

df_with_rolls = df_all.merge(sql_roll, on='license_number', suffixes=('_x', ''), how='left').drop(columns=['roll_up_id_x', 'company_roll_up_x'])

In [ ]:
df_with_rolls.head()

In [ ]:
sql_contact = pd.read_sql('co_contact', con=engine_con)

df_with_contacts = df_with_rolls.merge(sql_contact, on='license_number', suffixes=('_x', ''), how='left').drop(columns=['contact_city', 'contact_zip'])
# [['license_number', 'contact_website', 'contact_email', 'contact_phone', 'contact_street', 'contact_city']]

In [ ]:
df_with_contacts

In [ ]:
df_with_contacts = df_with_contacts.rename(columns={'name_legal_x':'name_legal','contact_city_x':'contact_city','contact_zip_x':'contact_zip'})

In [ ]:
df_with_contacts.columns

In [ ]:
df_with_contacts['contact_state'] = df_with_contacts['contact_state'].fillna('CO')

In [ ]:
df_with_contacts['contact_zip'] = df_with_contacts['contact_zip'].astype(int)

In [ ]:
df_with_contacts

### Reorder

In [ ]:
# Reordering these so that the roll up information is next to the name_legal and name_dba

df_reordered = df_with_contacts[[
    'license_number', 'license_category', 'license_description', 'name_legal', 'name_dba',
    'roll_up_id', 'company_roll_up',
    'contact_website', 'contact_email', 'contact_phone',
#     'contact_street',
    'contact_city', 'contact_zip',  'zip_longitude', 'zip_latitude',
    'contact_state', 'is_current', 'date_uploaded'
                ]]

In [ ]:
df_reordered.columns

### Move to CSV to Add Roll-Up Info

In [ ]:
df_reordered.sort_values(by='name_legal', inplace=True)

In [ ]:
# Convert the file to CSV so the company_roll_ups can be edited

df_reordered.to_csv(f'edited_files/df_{this_year}_{this_month_num}_to_edit.csv', index=False)

###### Make Changes to File Before Reading Back In

In [ ]:
# Convert the file back to a pandas dataframe

if pd.read_csv(f'edited_files/df_{this_year}_{this_month_num}_to_edit.csv').company_roll_up.isnull().sum() == 0:
    df_edited = pd.read_csv(f'edited_files/df_{this_year}_{this_month_num}_to_edit.csv')
else:
    print(error)
# df_edited = pd.read_csv('edited_files/df_first.csv')

In [ ]:
# Just for the 'original' csv
df_edited.rename(columns={'LicenseType':'license_description'}, inplace=True)

In [ ]:
# My little failsafe to make sure I didn't forget to add company roll ups for new companies

if '' in df_edited['company_roll_up']:
    print(error)

In [ ]:
df_edited.head()

In [ ]:
# This creates a list of the newly added company_roll_ups

new_comp = list(df_edited[df_edited['roll_up_id'].isnull()]['company_roll_up'].unique())

In [ ]:
# This creates the list of roll_up_ids that will be assigned to the new company_roll_ups

max_roll = max(df_edited.roll_up_id.str[3:].fillna('0').astype(int))

new_roll = []
for comp in new_comp:
    max_roll += 1
    new_roll.append(max_roll)
    
new_co_roll = []
for num in new_roll:
    num = str(num)
    while len(num) < 5:
        num = '0' + num
    new_co_roll.append(f'CO-{num}')

In [ ]:
new_co_roll

In [ ]:
# This creates two seperate dataframes for rows needing roll_up_ids and those that don't

df_edited_null = df_edited[df_edited['roll_up_id'].isnull()]
df_edited_fill = df_edited[df_edited['roll_up_id'].notnull()]

In [ ]:
# This uses the two lists to add roll_up_ids to the df_edited

for n in range(len(df_edited_null)):
    df_edited_null['roll_up_id'][n:n+1] = new_co_roll[new_comp.index(df_edited_null[n:n+1]['company_roll_up'].item())]

In [ ]:
df_edited_null

In [ ]:
# This reattaches the two dataframes now that the roll_up_ids have been filled

df_populated = df_edited_fill.append(df_edited_null)

In [ ]:
df_populated

### Converting License Types to Columns

In [ ]:
# This just takes the list of unique license types and changes them to the correct format

# all_license_types = list(df_populated.license_description.unique())

# license_columns = []

# for lic in all_license_types:
#     license_columns.append(f"license_{lic.lower().replace(' ', '_')}")
    
# license_columns

In [ ]:
# This function should properly fill out the license_columns with binary data

# def fill_lic_col(col):
#     for n in range(len(df_populated)):
#         df_populated[col][n] = int(df_populated['license_description'][n] == col.replace("_", " ").title()[8:])

In [ ]:
# This creates and fills the license_type columns as needed

# for col in license_columns:
#     df_populated[col] = 0
#     fill_lic_col(col)

In [ ]:
# Just checking that there numbers still add up the same
# df_populated.license_description.value_counts()

In [ ]:
# # Looks good
# for col in license_columns:
#     print(col, sum(df_populated[col]))

## SQL

In [ ]:
df_populated.columns

In [ ]:
# This creates a file similar to ca_main that can be added for Colorado

df_sql_main = df_populated[[
              'license_number', 'license_category', 'license_description', 'name_legal', 'name_dba', 'is_current', 'date_uploaded'
                            ]]

In [ ]:
# This creates a file similar to ca_roll that can be added for Colorado

df_sql_roll = df_populated[['license_number', 'roll_up_id', 'company_roll_up']]

In [ ]:
# This creates a file similar to ca_contact that can be added for Colorado

df_sql_contact = df_populated[[
                    'license_number', 'contact_website', 'contact_email', 'contact_phone', 
#                     'contact_street', 
                    'contact_city', 'contact_state', 'contact_zip'
                                ]]

In [ ]:
df_sql_contact

### Compare to SQL

In [ ]:
# This reads in the sql data to compare to the new data

sql_main = pd.read_sql('co_main', con=engine_con)
sql_roll = pd.read_sql('co_roll', con=engine_con)
sql_contact = pd.read_sql('co_contact', con=engine_con)

In [ ]:
# # This makes all of the old licenses blank
# sql_main['is_current'] =  0
# for n in range(len(sql_main)):
#     if sql_main['license_number'][n] in list(df_sql_main['license_number'].unique()):
#         sql_main['is_current'][n] = 1

In [ ]:
# And quickly make sure this is the correct type
df_sql_main['date_uploaded'] = pd.to_datetime(df_sql_main['date_uploaded'])

In [ ]:
# And a quick reorder
sql_main = sql_main[['license_number', 'license_category', 'license_description',
       'name_legal', 'name_dba', 'is_current', 'date_uploaded']]

In [ ]:
sql_main.shape

In [ ]:
df_sql_main.shape

In [ ]:
sql_roll.shape

In [ ]:
df_sql_roll.shape

In [ ]:
sql_contact.shape

In [ ]:
df_sql_contact.shape

In [ ]:
sql_main[sql_main['is_current'] == 0]

In [ ]:
# # This clears the licenses from df_sql_main that were already in sql

# for n in range(len(df_sql_main)):
#     if df_sql_main['license_number'][n] in list(sql_main.license_number.unique()):
#         df_sql_main.drop(index=n, inplace=True)

In [ ]:
df_sql_main.dtypes == sql_main.dtypes

In [ ]:
# Just gonna take out all the null values before comparing the two dataframes
df_sql_main['name_dba'] = df_sql_main['name_dba'].fillna('')

In [ ]:
df_sql_main[df_sql_main['is_current'] == 1]

In [ ]:
df_sql_roll.head()

In [ ]:
missing_lic = []

for lic in list(sql_main['license_number']):
    if lic not in list(df_sql_main['license_number']):
        missing_lic.append(lic)

In [ ]:
missing_lic_df = sql_main[sql_main['license_number'].isin(missing_lic)]

In [ ]:
missing_lic_df['is_current'] = 0

In [ ]:
missing_lic_df

In [ ]:
df_sql_main = df_sql_main.append(missing_lic_df).sort_values(by='name_legal').reset_index().drop(columns='index')

### Making Changes to the SQL Database

#### Main Alter

In [ ]:
# Start by compiling the data that needs to be checked
df_for_alter = df_sql_main

df_main_list = list(df_sql_main.license_number.unique())
sql_main_list = list(sql_main.license_number.unique())
not_current_main_list = []
for lic in sql_main_list:
    if lic not in df_main_list:
        not_current_main_list.append(lic)
        
not_current_df = pd.DataFrame(not_current_main_list, columns=['license_number']).merge(sql_main, how='left')

df_for_alter = df_for_alter.append(not_current_df)

df_for_alter = df_for_alter.reset_index().drop(columns=['index'])

In [ ]:
# for lic in df_for_alter[df_for_alter.is_current == 0]['license_number'].unique():
#     if list(sql_main[sql_main['license_number'] == lic]['is_current'])[0] == 1:
#         print(lic)

In [ ]:
df_sql_main[df_sql_main['license_number'] == '402-00085']

In [ ]:
# This creates a list of all of the license_numbers of rows that have a change in them

to_alter = []
no_longer_active = []

# There's probably a cleaner way to do this, but this gets the list of all licenses where there is some change in information
for lic in sql_main['license_number']:
    if list(df_sql_main[df_sql_main['license_number'] == lic]['license_category'])[0] != list(sql_main[sql_main['license_number'] == lic]['license_category'])[0]:
        print(lic, 'category')
        to_alter.append(lic)
    elif list(df_sql_main[df_sql_main['license_number'] == lic]['license_description'])[0] != list(sql_main[sql_main['license_number'] == lic]['license_description'])[0]:
        print(lic, 'description')
        to_alter.append(lic)
    elif list(df_sql_main[df_sql_main['license_number'] == lic]['is_current'])[0] != list(sql_main[sql_main['license_number'] == lic]['is_current'])[0]:
        print(lic, 'current')
        no_longer_active.append(lic)
        to_alter.append(lic)
        
# This may be only needed the first time
for x in range(len(df_for_alter)):
    if df_for_alter.loc[x]['is_current'] == 0:
        to_alter.append(df_for_alter.loc[x]['license_number'])

In [ ]:
# And then we turn that list into a DataFrame
main_alter = pd.DataFrame(list(to_alter), columns=['license_number']).merge(df_for_alter, how='left')

In [ ]:
main_alter['date_no_longer_active'] = ''

In [ ]:
for n in range(len(main_alter)):
    if main_alter['license_number'][n] in no_longer_active:
        main_alter['date_no_longer_active'][n] = month_year

In [ ]:
# And let's toss this into a csv 

# main_alter.to_excel(f'edited_files/main_alter_{this_month_num}_{this_year}.xlsx', sheet_name='co_main_alter', index=False)

In [ ]:
main_alter.to_sql(name='co_main_alter', con=engine_con, index=False, if_exists='replace')

In [ ]:
# This code runs the query noramally run in SQL to update co_main using co_main_alter

co_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=co_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

co_cursor = co_conn.cursor()

co_cursor.execute('SELECT * FROM co_cannabis.dbo.co_main')

co_cursor.execute('''
Update t1
SET t1.[license_description] = t2.[license_description],
    t1.[is_current] = t2.[is_current],
    t1.[date_no_longer_active] = t2.[date_no_longer_active]
FROM [co_cannabis].[dbo].[co_main] as t1
INNER JOIN [co_cannabis].[dbo].[co_main_alter] as t2
ON t1.[license_number] = t2.[license_number];
                  ''')

co_conn.commit()

#### Contact Alter

In [ ]:
sql_contact = pd.read_sql('co_contact', con=engine_con)

In [ ]:
# This creates a df of all the lciense numbers that will need to be altered

con_to_alter = []

for lic in list(df_sql_contact['license_number'].unique()):
    if list(df_sql_contact[df_sql_contact['license_number'] == lic]['contact_zip'])[0] != list(sql_contact[sql_contact['license_number'] == lic]['contact_zip'])[0]:
        print(lic, 'zip')
        con_to_alter.append(lic)
        
co_contact_for_alter = df_sql_contact[df_sql_contact['license_number'].isin(con_to_alter)]

In [ ]:
co_contact_for_alter = co_contact_for_alter.reset_index().drop(columns='index')

In [ ]:
# This adds the Longitude and Latitude for each license

co_contact_for_alter['zip_longitude'] = ''
co_contact_for_alter['zip_latitude'] = ''

nomi = pgeocode.Nominatim('us')

for n in range(len(co_contact_for_alter)):
    zipcode = str(co_contact_for_alter['contact_zip'][n])
    if len(zipcode) > 5:
        zipcode = zipcode[:5]
    co_contact_for_alter['zip_longitude'][n] = nomi.query_postal_code(zipcode)['longitude']
    co_contact_for_alter['zip_latitude'][n] = nomi.query_postal_code(zipcode)['latitude']

In [ ]:
co_contact_for_alter.to_sql('co_contact_alter', con=engine_con, if_exists='replace', index=False)

In [ ]:
# This code runs the query noramally run in SQL to update co_main using co_contact_alter

co_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=co_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

co_cursor = co_conn.cursor()

co_cursor.execute('SELECT * FROM co_cannabis.dbo.co_contact')

co_cursor.execute('''
Update t1
SET t1.[contact_zip] = t2.[contact_zip],
    t1.[zip_longitude] = t2.[zip_longitude],
    t1.[zip_latitude] = t2.[zip_latitude]
FROM [co_cannabis].[dbo].[co_contact] as t1
INNER JOIN [co_cannabis].[dbo].[co_contact_alter] as t2
ON t1.[license_number] = t2.[license_number];
                  ''')

co_conn.commit()

### Cleaning Data for SQL

In [ ]:
# This clears the licenses from df_sql_roll that were already in sql

for n in range(len(df_sql_roll)):
    if df_sql_roll['license_number'][n] in list(sql_roll.license_number.unique()):
        df_sql_roll.drop(index=n, inplace=True)

In [ ]:
# This clears the licenses from df_sql_contact that were already in sql

for n in range(len(df_sql_contact)):
    if df_sql_contact['license_number'][n] in list(sql_contact.license_number.unique()):
        df_sql_contact.drop(index=n, inplace=True)

In [ ]:
# This clears the licenses from sql_main that were already in sql

for n in range(len(df_sql_main)):
    if df_sql_main['license_number'][n] in list(sql_main.license_number.unique()):
        df_sql_main.drop(index=n, inplace=True)

In [ ]:
df_sql_roll

## To SQL

#### co_main

In [ ]:
df_sql_main.to_sql(name='co_main', con=engine_con, index=False, if_exists='append')

#### co_roll

In [ ]:
df_sql_roll.to_sql(name='co_roll', con=engine_con, index=False, if_exists='append')

#### co_contact

In [ ]:
df_sql_contact = df_sql_contact.reset_index().drop(columns='index')

In [ ]:
# This adds the Longitude and Latitude for each license

df_sql_contact['zip_longitude'] = ''
df_sql_contact['zip_latitude'] = ''

nomi = pgeocode.Nominatim('us')

for n in range(len(df_sql_contact)):
    zipcode = str(df_sql_contact['contact_zip'][n])
    if len(zipcode) > 5:
        zipcode = zipcode[:5]
    df_sql_contact['zip_longitude'][n] = nomi.query_postal_code(zipcode)['longitude']
    df_sql_contact['zip_latitude'][n] = nomi.query_postal_code(zipcode)['latitude']

In [ ]:
df_sql_contact.to_sql(name='co_contact', con=engine_con, index=False, if_exists='append')

In [ ]:
sql_main = pd.read_sql('co_main', con=engine_con)
sql_contact = pd.read_sql('co_contact', con=engine_con)

In [ ]:
sql_main_contact = sql_main.merge(sql_contact, how='outer', on='license_number')

In [ ]:
sql_main_contact[sql_main_contact['is_current'] == 0]

### all_main

In [ ]:
# This creates the connection to the all_main file in SQL

all_cannabis_params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=all_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
all_cannabis_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(all_cannabis_params))

In [ ]:
# This reads in the all_main data and narrows it down to the relevent data
all_main_sql = pd.read_sql('all_main', con=all_cannabis_con)
co_main_sql = all_main_sql[all_main_sql['us_state'] == 'CO']
co_main_sql = co_main_sql.reset_index().drop(columns='index')

In [ ]:
# This re-reads in and merges the co_annabis data in SQL
updated_co_main = pd.read_sql('co_main', con=engine_con)
updated_co_roll = pd.read_sql('co_roll', con=engine_con)
updated_co_cannabis = updated_co_main.merge(updated_co_roll, on='license_number')

In [ ]:
updated_co_cannabis['license_status'] = ''

In [ ]:
updated_co_cannabis['license_sub_category'] = ''
for n in range(len(updated_co_cannabis)):
    if updated_co_cannabis['license_description'][n] in ['Medical Cultivation', 'Retail Cultivation']:
        updated_co_cannabis['license_sub_category'][n] = 'Cultivator'
    if updated_co_cannabis['license_description'][n] in ['Medical Store', 'Retail Store', 'Medical Operator', 'Retail Operator']:
        updated_co_cannabis['license_sub_category'][n] = 'Retailer'
    if updated_co_cannabis['license_description'][n] in ['Medical Product Manufacturer', 'Retail Product Manufacturer']:
        updated_co_cannabis['license_sub_category'][n] = 'Manufacturer'
    if updated_co_cannabis['license_description'][n] in ['Medical Transporter', 'Retail Transporter', 'Medical Delivery', 'Retail Delivery']:
        updated_co_cannabis['license_sub_category'][n] = 'Distributor'
    if updated_co_cannabis['license_description'][n] in ['Medical Research Development Cultivation', 'Medical Hospitality']:
        updated_co_cannabis['license_sub_category'][n] = 'Other'

In [ ]:
new_all_main = updated_co_cannabis[['license_number', 'roll_up_id', 'name_legal', 'license_status', 'is_current', 'license_description', 'license_sub_category']]
new_all_main['state'] = 'CO'

In [ ]:
# This takes out everything already there
new_main_for_sql = new_all_main[~new_all_main['license_number'].isin(list(co_main_sql['license_number'].unique()))]

In [ ]:
new_main_for_sql = new_main_for_sql.rename(columns={'roll_up_id':'rollup_id', 'name_legal':'legal_name', 'state':'us_state', 'license_sub_category':'license_category'})
new_main_for_sql['expiration_date'] = None
new_main_for_sql.head()

In [ ]:
new_main_for_sql.shape

In [ ]:
# And this uploads the new licenses to SQL
new_main_for_sql.to_sql('all_main', con=all_cannabis_con, index=False, if_exists='append')

#### Alter

In [ ]:
# This adds the zipcode info for the new_all_main dataframe
new_all_main = new_all_main.merge(sql_contact[['license_number', 'contact_zip', 'zip_longitude', 'zip_latitude']], how='left', on='license_number')

In [ ]:
new_all_main['contact_zip'] = new_all_main['contact_zip'].astype(int).astype(str)

In [ ]:
# This reads in the all_main data and narrows it down to the relevent data
all_main_sql_for_alter = pd.read_sql('all_main', con=all_cannabis_con)
co_main_sql_for_alter = all_main_sql_for_alter[all_main_sql_for_alter['us_state'] == 'CO']
co_main_sql_for_alter = co_main_sql_for_alter.reset_index().drop(columns='index')

In [ ]:
co_main_sql_for_alter

In [ ]:
# This creates a df of all the lciense numbers that will need to be altered

co_lic_to_alter = []

for lic in list(new_all_main['license_number'].unique()):
    if list(new_all_main[new_all_main['license_number'] == lic]['is_current'])[0] != list(co_main_sql_for_alter[co_main_sql_for_alter['license_number'] == lic]['is_current'])[0]:
        print(lic, 'current')
        co_lic_to_alter.append(lic)
    elif list(new_all_main[new_all_main['license_number'] == lic]['roll_up_id'])[0] != list(co_main_sql_for_alter[co_main_sql_for_alter['license_number'] == lic]['rollup_id'])[0]:
        print(lic, 'roll')
        co_lic_to_alter.append(lic)
    elif list(new_all_main[new_all_main['license_number'] == lic]['license_status'])[0] != list(co_main_sql_for_alter[co_main_sql_for_alter['license_number'] == lic]['license_status'])[0]:
        print(lic, 'status')
        co_lic_to_alter.append(lic)
    elif list(new_all_main[new_all_main['license_number'] == lic]['license_description'])[0] != list(co_main_sql_for_alter[co_main_sql_for_alter['license_number'] == lic]['license_description'])[0]:
        print(lic, 'description')
        co_lic_to_alter.append(lic)
    elif list(new_all_main[new_all_main['license_number'] == lic]['license_sub_category'])[0] != list(co_main_sql_for_alter[co_main_sql_for_alter['license_number'] == lic]['license_category'])[0]:
        print(lic, 'category')
        co_lic_to_alter.append(lic)
    elif list(new_all_main[new_all_main['license_number'] == lic]['contact_zip'])[0] != list(co_main_sql_for_alter[co_main_sql_for_alter['license_number'] == lic]['contact_zip'])[0]:
        print(lic, 'zip')
        co_lic_to_alter.append(lic)
    
        
co_all_main_for_alter = new_all_main[new_all_main['license_number'].isin(co_lic_to_alter)]

In [ ]:
co_all_main_for_alter

In [ ]:
co_all_main_for_alter.to_sql('co_main_alter', con=all_cannabis_con, if_exists='replace', index=False)

In [ ]:
# This code runs the query noramally run in SQL to update co_main using co_contact_alter

all_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=all_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

all_cursor = all_conn.cursor()

all_cursor.execute('SELECT * FROM co_cannabis.dbo.co_contact')

all_cursor.execute('''
Update t1
SET t1.[is_current] = t2.[is_current],
    t1.[rollup_id] = t2.[roll_up_id],
    t1.[license_status] = t2.[license_status],
    t1.[license_description] = t2.[license_description],
    t1.[license_category] = t2.[license_sub_category],
    t1.[contact_zip] = t2.[contact_zip],
    t1.[zip_longitude] = t2.[zip_longitude],
    t1.[zip_latitude] = t2.[zip_latitude]
FROM [all_cannabis].[dbo].[all_main] as t1
INNER JOIN [all_cannabis].[dbo].[co_main_alter] as t2
ON t1.[license_number] = t2.[license_number];
                  ''')

all_conn.commit()

### Bespoke Platform

In [ ]:
new_main

In [ ]:
new_main = new_main_for_sql.append(co_all_main_for_alter)

In [ ]:
new_main['expiration_date'] = None

In [ ]:
new_main = new_main.reset_index().drop(columns='index')

In [ ]:
new_main

In [ ]:
def send_to_platform(data, production=bool):
    if production == 1:
        requests.post(
        url='https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
        json={'company_licenses': data},
        headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'})
    elif production == 0:
        requests.post(
        url='https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses',
        json={'company_licenses': data},
        headers={'x-api-key': '80030179832729980933'})
    

n = 0

while len(new_main) - n > 0:
    
    list_for_platform = []
    
    while len(list_for_platform) < 25 and n < len(new_main):
    
        if new_main['is_current'].iloc[n] == 1:
            is_current = True
        else:
            is_current = False

        list_for_platform.append({
        'license_number': new_main['license_number'][n],
        'rollup_id': new_main['rollup_id'][n],
        'legal_name': new_main['legal_name'][n],
        'license_status': new_main['license_status'][n],
        'is_current': is_current,
        'license_description': new_main['license_description'][n],
        'us_state': 'CO',
        'expiration_date': None,
        'license_category': new_main['license_category'][n],
        'estimate_zip': new_main['contact_zip'][n],
        'estimate_longitude': new_main['zip_longitude'][n],
        'estimate_latitude': new_main['zip_latitude'][n]
            
        })
        
        n+=1
        
        #Change below to Boolian
        
    send_to_platform(list_for_platform, production=1)
    
    print(n)
    
    sleep(5)

In [ ]:
new_main

In [ ]:
print(stop)

### Check the Platform

In [ ]:
current_metabase_df = pd.read_json('query_result_2022-01-07T18_32_17.92Z.json')

In [ ]:
all_main_df = pd.read_sql('all_main', all_cannabis_con)
co_all_main_df = all_main_df[all_main_df['us_state'] == 'CO']

In [ ]:
for lic in list(co_all_main_df['license_number']):
    if lic not in list(current_metabase_df['License Number']):
        print(lic)